In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
from tqdm import tqdm
import xgboost as xgb
import tensorflow as tf
from keras.applications.densenet import DenseNet121
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Flatten, Input, Dense

import scipy
from sklearn.metrics import fbeta_score
import csv

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output


random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

n_classes = 7

train_path = "/content/drive/MyDrive/CasmeIILabeledZara"
train = pd.read_csv("/content/drive/MyDrive/casmeII_zara.csv")

#csvPath = os.path.sep.join(['/content/drive/MyDrive/',
 # "{}.csv".format('exp-densenet-features')])
#csvFile = open(csvPath, "w")

#writer = csv.writer(csvFile)

# use ResNet50 model extract feature from fc1 layer
base_model = DenseNet121(weights='imagenet', pooling=max, include_top = False)
input = Input(shape=(224,224,3),name = 'image_input')
x = base_model(input)
x = Flatten()(x)
x = Dense(1000)(x)
model = Model(inputs=input, outputs=x)

count = 0

total_img_count = 1
#for folder, f in tqdm(train.values[:100]):
for chunk in pd.read_csv("/content/drive/MyDrive/casmeII_zara.csv", chunksize=100):

  #if(count > 0):
  # header = False

  #17118
  if(total_img_count > 10):
    break
  
  X_train = []

  print('chunk no'+str(count)+'\n')
  print('img no'+str(total_img_count)+'\n')

  for folder, f in tqdm(chunk.values[:len(chunk)]):
    #print(folder+'/'+f+'\n')

    img_path = train_path + "/{}/{}".format(folder,f)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    features_reduce =  features.squeeze()
    #print(features_reduce.shape)
    X_train.append(np.append([folder,f],features_reduce))

    total_img_count = total_img_count + 1

  train_ResNet =  pd.DataFrame(np.array(X_train))
  #train_ResNet = pd.concat([chunk, train_ResNet], axis = 1)
  train_ResNet.to_csv('/content/drive/MyDrive/test-exp-features-densenet_casmeII_.csv', mode='a', header=False, index=False)
  #train_ResNet.to_csv('train_ResNet_Feature.csv', mode='a', header=False, index=False)

  count = count+1
  
    #writer.writerow(np.append(folder,features_reduce))



29089792/29084464 [==============================] - 0s 0us/step


  0%|          | 0/100 [00:00<?, ?it/s]

chunk no0

img no1



100%|██████████| 100/100 [00:47<00:00,  2.12it/s]
